## RAG Pipeline

In [4]:
import openai
from qdrant_client import QdrantClient

In [5]:
qdrant_client = QdrantClient(url="http://localhost:6333")

#### Embedding Function

In [6]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(input=text, model="text-embedding-3-small")
    return response.data[0].embedding

#### Qdrant Retrieval Function

In [11]:
def retrieve_data(query, k=5):
    query_embedding = get_embeddings(query)
    results = qdrant_client.query_points(
        collection_name="amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids, 
        "retrieved_context": retrieved_context, 
        "similarity_scores": similarity_scores
    }


    

In [13]:
response = retrieve_data("What kind of Earphones can I get", k=5)
response

    

{'retrieved_context_ids': ['B09F36P17Y',
  'B0BG29CCQ3',
  'B09TKF5S6W',
  'B0BS1GQJ5S',
  'B09MQBZC2B'],
 'retrieved_context': ['Wireless Earbud, Bluetooth 5.1 Headphones with Microphone Deep Bass Bluetooth Earphones in-Ear, CVC8.0 Noise Cancelling Earbud for Sport Running Gym IPX7 Waterproof, 30H Playtime, Touch Control White ',
  'USB C Earphone for Google Pixel 7 Pro 7A6 6A, Type C Headphones in Ear Wired Earbuds Magnetic Noise Canceling Headset Microphone for iPad 10 Samsung Galaxy S23 S22 Ultra S21 FE S20 A54 Oneplus Green USB C Headphone: Widely compatible with Samsung Galaxy S23 Ultra 2023/S23 Plus S23+/S23/S22 Ultra/S22 Plus S22+/S22/S21 FE/S21/S21+/S21 Ultra/S20 FE/S20/S20 Ultra/S20 Plus/Galaxy Z Fold 4 3/Galaxy Z Flip 4 3/A53/A54/A33/A34, for iPad 10,iPad Air 4th/5th, iPad Pro 2022/2021/2020/ 2018/iPad Mini 6th, for MacBook 12"/MacBook Air 13"/MacBook Pro 13"/14"/15"/16",for Google Pixel 7 Pro/7/7A/6A/6 Pro/6/5,for OnePlus 11 2023/10T 5G/10 Pro/9 Pro/9/8T/8 Pro/7T Pro/7T/7 P

#### Format Retrieved Context

In [19]:
def process_context(context):
    formatted_context = ""
    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):
        formatted_context += f"- {id}: {chunk}\n"
    return formatted_context

In [23]:
preprocessed_context = process_context(response)
print(preprocessed_context)

- B09F36P17Y: Wireless Earbud, Bluetooth 5.1 Headphones with Microphone Deep Bass Bluetooth Earphones in-Ear, CVC8.0 Noise Cancelling Earbud for Sport Running Gym IPX7 Waterproof, 30H Playtime, Touch Control White 
- B0BG29CCQ3: USB C Earphone for Google Pixel 7 Pro 7A6 6A, Type C Headphones in Ear Wired Earbuds Magnetic Noise Canceling Headset Microphone for iPad 10 Samsung Galaxy S23 S22 Ultra S21 FE S20 A54 Oneplus Green USB C Headphone: Widely compatible with Samsung Galaxy S23 Ultra 2023/S23 Plus S23+/S23/S22 Ultra/S22 Plus S22+/S22/S21 FE/S21/S21+/S21 Ultra/S20 FE/S20/S20 Ultra/S20 Plus/Galaxy Z Fold 4 3/Galaxy Z Flip 4 3/A53/A54/A33/A34, for iPad 10,iPad Air 4th/5th, iPad Pro 2022/2021/2020/ 2018/iPad Mini 6th, for MacBook 12"/MacBook Air 13"/MacBook Pro 13"/14"/15"/16",for Google Pixel 7 Pro/7/7A/6A/6 Pro/6/5,for OnePlus 11 2023/10T 5G/10 Pro/9 Pro/9/8T/8 Pro/7T Pro/7T/7 Pro/7/5/5T and more USB C devices Magnetic USB C Headset: Each earbud has a powerful magnetic embedded into 

#### Create Prompt Function

In [22]:
def build_prompt(preprocessed_context, question):
    prompt = f"""
You are a shopping assistant that can answer questions about the productsin stock. You will be give a question and the list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use the word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""
    return prompt


In [26]:
## Test
formatted_prompt = build_prompt(preprocessed_context, "What kind of Earphones can I get")
print(formatted_prompt)


You are a shopping assistant that can answer questions about the productsin stock. You will be give a question and the list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use the word context and refer to it as the available products.

Context:
- B09F36P17Y: Wireless Earbud, Bluetooth 5.1 Headphones with Microphone Deep Bass Bluetooth Earphones in-Ear, CVC8.0 Noise Cancelling Earbud for Sport Running Gym IPX7 Waterproof, 30H Playtime, Touch Control White 
- B0BG29CCQ3: USB C Earphone for Google Pixel 7 Pro 7A6 6A, Type C Headphones in Ear Wired Earbuds Magnetic Noise Canceling Headset Microphone for iPad 10 Samsung Galaxy S23 S22 Ultra S21 FE S20 A54 Oneplus Green USB C Headphone: Widely compatible with Samsung Galaxy S23 Ultra 2023/S23 Plus S23+/S23/S22 Ultra/S22 Plus S22+/S22/S21 FE/S21/S21+/S21 Ultra/S20 FE/S20/S20 Ultra/S20 Plus/Galaxy Z Fold 4 3/Galaxy Z Flip 4 3/A53/A54/A33/A34, for iPad 10,iPad Air 4th/5th, iPad Pro 2022

### Generate LLM Answer Function

In [28]:
def generate_answer(prompt):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )
    return response.choices[0].message.content


In [29]:
generated_answer = generate_answer(formatted_prompt)

In [30]:
print(generated_answer)

You can choose from the following types of earphones:

1. Wireless Earbuds:
   - B09F36P17Y: Bluetooth 5.1 wireless earbuds with deep bass, CVC8.0 noise cancelling, IPX7 waterproof, 30 hours playtime, touch control, suitable for sports.
   - B09TKF5S6W: True wireless earbuds with 30 hours playtime, LED display, wireless charging case, IPX4 waterproof, Bluetooth 5.0, touch control, suitable for iPhone, Samsung, Android, and laptops.
   - B09MQBZC2B: NYANDU Bluetooth 5.1 true wireless earbuds with noise cancelling, 40 hours playtime, USB-C charging, IPX5 waterproof, touch control, extra bass, comfortable fit.

2. Wired USB-C Earphones:
   - B0BG29CCQ3: USB C wired earphones compatible with Google Pixel, Samsung Galaxy, iPad, MacBook, OnePlus, with magnetic earbuds, noise cancelling microphone, remote control, ergonomic silicone ear tips, and carrying bag.

3. Wired Earbuds:
   - B0BS1GQJ5S: Togkun wired earbuds with built-in mic, comfortable in-ear design.

These options include wireless

### Combined RAG Functions into a Pipeline

In [33]:
def rag_pipeline(query, top_k=5):
    """
    1. Embed Query
    2. Perform Search against Vector DB
    """
    qdrant_client = QdrantClient(url="http://localhost:6333")
    retrieved_context = retrieve_data(query, k=top_k)
    preprocessed_context = process_context(retrieved_context)
    formatted_prompt = build_prompt(preprocessed_context, query)
    generated_answer = generate_answer(formatted_prompt)
    return generated_answer

In [ ]:
answer = rag_pipeline("What kind of Earphones can I get", top_k=10)
print(answer)

You can choose from several types of earphones available:

1. Wireless Bluetooth Earbuds:
   - B09F36P17Y: Wireless Earbuds with Bluetooth 5.1, deep bass, CVC8.0 noise cancelling, IPX7 waterproof, 30 hours playtime, touch control, suitable for sports.
   - B09TKF5S6W: True Wireless Earbuds with Bluetooth 5.0, 30 hours total playtime, wireless charging case, IPX4 waterproof, LED battery display, touch control.
   - B09MQBZC2B: NYANDU Bluetooth 5.1 True Wireless Earbuds with noise cancelling, 40 hours playtime, IPX5 waterproof, touch control.
   - B0BJ9PRHZ3: Soundcore by Anker Space A40 Active Noise Cancelling Wireless Earbuds, 50 hours playtime, Hi-Res sound, wireless charging.
   
2. Wired USB-C Earphones:
   - B0BG29CCQ3: USB C wired earphones compatible with Google Pixel, Samsung Galaxy, iPad, OnePlus, featuring magnetic earbuds, noise cancelling microphone, remote control, ergonomic design.

3. Wired Earbuds:
   - B0BS1GQJ5S: Togkun 2 Packs wired earbuds with built-in microphone, c